In [ ]:
import os
os.getcwd()

In [ ]:
os.chdir("../../..")
os.getcwd()

In this tutorial, you’re going to use a Jupyter notebook and the ISAAC Python API. 

The first thing you do is import Isaac and create an application.

Click on the cell each cell in order and press the `Run` button above to see the code in action.

In [ ]:
# Create an Isaac application
from engine.pyalice import Application, Codelet, Message
app = Application()
print('Application created')

Now configure your application graph. Your sample application will have a graph with two nodes: 
1. A “mock” node that continually generates mock camera and object detection data.
2. An “echo” node that is configured to echo back any messages sent to it.

Start by creating and configuring the mock node.

In [ ]:
# Load the message generator module
app.load_module('message_generators')

# Create the mock node
mock_node = app.add('mock')

# Add the camera component to the mock node
mock_camera = mock_node.add(app.registry.isaac.message_generators.CameraGenerator, 'camera')
mock_camera.config['tick_period'] = '50ms'
mock_camera.config['rows'] = 240
mock_camera.config['cols'] = 320

# Add the detection component to the mock node
mock_detection = mock_node.add(app.registry.isaac.message_generators.Detections2Generator, 'detections2')
mock_detection.config['tick_period'] = '50ms'
mock_detection.config['detection_configuration'] = [
  {
    "class_label": "mock_detection",
    "confidence": 0.8,
    "bounding_box_coordinates": [
      0.0,
      0.0,
      100.0,
      100.0
    ]
  }
]

print('Mock node created')

Next, create the echo node and connect the `in` label to the `out` label.

In [ ]:
# Create the echo node
echo_node = app.add('echo')

# Get the MessageLedger from the echo node. 
message_ledger = echo_node['MessageLedger']

# Connect the `in` label to the `out` label. This will cause any messages sent to `in` to be rebroadcasts to `out`
app.connect(message_ledger, 'in', message_ledger, 'out')

print('Echo node created')

Start the application.

In [ ]:
app.start()

Grab an image from the mock camera. You must import the necessary libraries. Then you ask to receive the latest message from the color_left channel in the camera component running in the mock node. Finally, you use PIL and matplotlib to display the image in your Jupyter notebook.

In [ ]:
# Receive and display a single camera image
from PIL import Image
from matplotlib.pyplot import imshow
msg = app.receive("mock", "camera", "color_left")
imshow(Image.fromarray((msg.tensor)))

Isaac uses Cap’n Proto and binary buffers to keep packing messages efficient. In the next example, you create a ping message, attach a byte buffer using numpy, and send it to the echo node.

In [ ]:
# Create a message and attach a buffer
import numpy as np
send_msg = Message.create_message_builder('PingProto')
send_msg.proto.message = 'Hello from Isaac'
send_msg.buffers = [np.arange(20, dtype=np.dtype('B'))]
app.publish('echo', 'MessageLedger', 'in', send_msg)
print('Message sent')

Because you sent a message to the `in` channel in the ledger component of the echo node, you can expect to receive the same message in the `out` channel of the ledger component of the echo node.

In [ ]:
# Receive a message, print the proto and buffer
recv_msg = app.receive('echo', 'MessageLedger', 'out')
print(recv_msg.proto.message)
print(np.asarray(recv_msg.buffers[0]))

Isaac supports a variety of message types. For more information, see [Message API Overview](https://docs.nvidia.com/isaac/isaac/doc/message_api.html).

You can quickly explore the payload of any message type by printing the .proto field. Here is an example using a detection message:

In [ ]:
# Discovering information about a proto payload
recv_msg = app.receive('mock', 'detections2', 'mock_detections')
print(recv_msg.proto)

Components may have configuration parameters that can be set at runtime. For example, the mock camera has “rows” and “cols” parameters that control the resolution of the mock camera.

In [ ]:
# Get config values of camera and display them
print(mock_camera.config['rows'])
print(mock_camera.config['cols'])

You can also change these configuration parameters

In [ ]:
# Set config values of camera to 720p, and display them
mock_camera.config['rows'] = 720
mock_camera.config['cols'] = 1280
print(mock_camera.config['rows'])
print(mock_camera.config['cols'])

Get an image from the mock camera and see that it’s now a 720p image.

In [ ]:
# Receive and display a single camera image, now at 720p
msg = app.receive("mock", "camera", "color_left")
imshow(Image.fromarray((msg.tensor)))

When you are done with the Isaac application, you should call stop. This lets Isaac know to safely stop and release any resources that it was using.

In [ ]:
# Stop the ISAAC application
app.stop()